# CPU + GPU Offloading
Install lama.cpp-python with cuBLAS:
```bash
CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python
```
or better, use ctransformers:
```
pip install ctransformers[cuda]
pip install ctransformers[gptq]
```


In [2]:
from ctransformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
#llm = AutoModelForCausalLM.from_pretrained("TheBloke/Yi-6B-200K-GGUF", model_file="yi-6b-200k.Q5_K_M.gguf", model_type="yi", gpu_layers=12) # hf=True
#llm = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.2-GGUF", model_file="mistral-7b-instruct-v0.2.Q5_K_M.gguf", model_type="mistral", gpu_layers=12) 
#llm = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-v0.1-GGUF", model_file="mistral-7b-v0.1.Q5_K_M.gguf", model_type="mistral", gpu_layers=8) 
llm = AutoModelForCausalLM.from_pretrained("TheBloke/phi-2-GGUF", model_file="phi-2.Q5_K_M.gguf", model_type="phi", gpu_layers=100) 

#tokenizer = AutoTokenizer.from_pretrained(llm)
#print(llm("AI is going to "))

Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 13231.24it/s]


RuntimeError: Failed to create LLM 'phi' from '/home/ruzickal/.cache/huggingface/hub/models--TheBloke--phi-2-GGUF/blobs/ce6bcc8fa75b467e1eb693bcc921fe366ceac14aa92ec3dc84da536c2289510a'.

In [1]:
from llama_cpp import Llama

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = Llama(
  model_path="/home/ruzickal/.cache/huggingface/hub/models--TheBloke--phi-2-GGUF/snapshots/5a454d977c6438bb9fb2df233c8ca70f21c87420/phi-2.Q5_K_M.gguf",  # Download the model file first
  n_ctx=2048,  # The max sequence length to use - note that longer sequence lengths require much more resources
  n_threads=8,            # The number of CPU threads to use, tailor to your system and the resulting performance
  n_gpu_layers=35         # The number of layers to offload to GPU, if you have GPU acceleration available
)

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Quadro M3000M, compute capability 5.2
llama_model_loader: loaded meta data with 20 key-value pairs and 325 tensors from /home/ruzickal/.cache/huggingface/hub/models--TheBloke--phi-2-GGUF/snapshots/5a454d977c6438bb9fb2df233c8ca70f21c87420/phi-2.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  2560, 51200,     1,     1 ]
llama_model_loader: - tensor    1:             blk.0.attn_norm.bias f32      [  2560,     1,     1,     1 ]
llama_model_loader: - tensor    2:           blk.0.attn_norm.weight f32      [  2560,     1,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_qkv.bias f32      [  7680,     1,     1,     1 ]
llama_model_loader: - tensor    4:            blk.0.attn_qkv.weight q6_K     [  2560,  7680,     1,     1 ]
llama_model_loader: - tensor    5:  

In [ ]:
# Simple inference example
output = llm(
  "Instruct: {prompt}\nOutput:", # Prompt
  max_tokens=512,  # Generate up to 512 tokens
  stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
  echo=True        # Whether to echo the prompt
)

In [12]:
stream = llm.create_chat_completion(
    messages = [
        {"role": "system", "content": "You are a story writing assistant."},
        {
            "role": "user",
            "content": "Write a story about llamas."
        }
    ],
    stream=True
)

result = []
for message in stream:
    result.append(message)
    print(message["choices"][0]["delta"])

Llama.generate: prefix-match hit


{'role': 'assistant'}
{'content': '\n'}
{'content': 'The'}
{'content': ' Story'}
{'content': ' of'}
{'content': ' the'}
{'content': ' Ll'}
{'content': 'ama'}
{'content': '\n'}
{'content': 'L'}
{'content': 'lam'}
{'content': 'as'}
{'content': ' are'}
{'content': ' one'}
{'content': ' of'}
{'content': ' the'}
{'content': ' most'}
{'content': ' interesting'}
{'content': ' animals'}
{'content': ' in'}
{'content': ' the'}
{'content': ' world'}
{'content': '.'}
{'content': ' They'}
{'content': ' have'}
{'content': ' been'}
{'content': ' domest'}
{'content': 'icated'}
{'content': ' for'}
{'content': ' thousands'}
{'content': ' of'}
{'content': ' years'}
{'content': ' and'}
{'content': ' are'}
{'content': ' used'}
{'content': ' for'}
{'content': ' transportation'}
{'content': ','}
{'content': ' wool'}
{'content': ' production'}
{'content': ','}
{'content': ' and'}
{'content': ' even'}
{'content': ' as'}
{'content': ' therapy'}
{'content': ' animals'}
{'content': '.'}
{'content': ' In'}
{'conte

KeyboardInterrupt: 

In [4]:
llm.n_vocab()

51200

In [16]:
import numpy as np
np.array(llm.eval_logits).shape

(1, 51200)

In [4]:
llm.pad_token_id

2

In [3]:
tokens = llm.tokenize("AI is going to ")
tokens = llm.prepare_inputs_for_generation(tokens)
llm.eval(tokens)
logits = np.array(llm.logits)
selection = np.argmax(logits, axis=-1)
llm.detokenize([selection])

' change'

In [9]:
np.array(llm.logits).shape

(64000,)

In [11]:
llm.config

Config(top_k=40, top_p=0.95, temperature=0.8, repetition_penalty=1.1, last_n_tokens=64, seed=-1, batch_size=8, threads=-1, max_new_tokens=256, stop=None, stream=False, reset=True, context_length=-1, gpu_layers=12, mmap=True, mlock=False)

In [ ]:
files